In [0]:
import numpy as np
from gensim.models import Word2Vec
import re
import json
import glob
import os
from sklearn.metrics.pairwise import cosine_similarity
import zipfile

In [0]:
basePath = '/content/'

data_dir = basePath+'dataset'
drive_dir = basePath+'drive/My Drive/temp/DatasetN'
paraFile = basePath + 'drive/My Drive/nlpproject/paragraph2.json'
"""
data_dir = basePath+'tempdataset'
drive_dir = basePath+'drive/My Drive/temp/tempDataset'
paraFile = basePath + 'drive/My Drive/temp.json'
"""
outDir = basePath+'test'

max_seq_length = 512

In [35]:
"""for file_name in os.listdir('/content/drive/My Drive'):
    if file_name.endswith('.zip'):
        with zipfile.ZipFile('/content/drive/My Drive/'+file_name,'r') as zip_dir:
            zip_dir.extractall(path='/content/')"""

"for file_name in os.listdir('/content/drive/My Drive'):\n    if file_name.endswith('.zip'):\n        with zipfile.ZipFile('/content/drive/My Drive/'+file_name,'r') as zip_dir:\n            zip_dir.extractall(path='/content/')"

In [0]:
def sifEmbeddings(sentences, model, alpha):

    vocab = model.wv.vocab
    embs = model.wv       
    size = model.vector_size  
    
    total = 0
    for word in vocab:
        total += vocab[word].count 

    output = []
    
    for sent in sentences:
        count = 0
        v = np.zeros(size, dtype=np.float32) 
        for word in sent:
          if word in vocab:
            #print(word)
            weight = alpha / (alpha + (vocab[word].count/total))
            v += weight * embs[word]
            count += 1 
                
        if count > 0:
            v *= 1/count

        output.append(v)
    return np.vstack(output)

In [0]:
def splitSent(sent):
    regX = '[\s()\.\?]'
    splitSent = re.split(regX,sent)
    splitSent = list(filter(None,splitSent))
    return splitSent

In [0]:
def getFileSent(inpFile):
    fileSent = ''
    article = ''
    with open(inpFile,'r') as f:
        x = json.loads(f.read())
        article = x['article']
        for option in x['options'][0]:
            fileSent += option
            fileSent += ' '
        fileSent += x['questions'][0]
    return fileSent,article

In [0]:
def getDatasetAsSents():
    fileSents = []
    dirs = os.listdir(data_dir)
    print(dirs)
    for d in dirs:
      filenames = glob.glob(os.path.join(data_dir,d)+"/*json")
      for file in filenames:
          with open(file,'r') as f:
              x = json.loads(f.read())
              for option in x['options'][0]:
                  fileSents.append(option)
              fileSents.append(x['questions'][0])
              fileSents.append(x['article'])
    return fileSents

In [0]:
def getQuesEmb(model,inpFile):
    fileSent,article = getFileSent(inpFile)
    fileSent = fileSent + (' ' + fileSent)*5
    sent = splitSent(fileSent)
    emb = sifEmbeddings([sent], model, alpha=alpha)[0]
    return emb,article

In [0]:
def saveFile(inpFile,outFile,para):
    with open(inpFile,'r') as f:
      x = json.loads(f.read())
      x['article'] = para

      with open(outFile, 'w') as jsonOut:
          json.dump(x, jsonOut)

In [42]:
fileSents = getDatasetAsSents()
sentences = [ splitSent(sent) for sent in fileSents]

['test', 'dev', 'train']


In [43]:
model = Word2Vec(sentences, min_count=1, size=200)
model.save(basePath+"word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
f = open(paraFile,'r')
x = json.loads(f.read())
f.close()
fileSents = [ x['para'+str(id)] for id in range(1,len(x)+1)]
sentences = [ splitSent(sent) for sent in fileSents]

In [45]:
#model = Word2Vec.load("word2vec.model")
alpha=0.001
paraEembs = sifEmbeddings(sentences, model, alpha=alpha)
print(paraEembs.shape)
np.savetxt(basePath+'paraEmbs.csv',paraEembs, delimiter=',',fmt='%.15f')

(12304, 200)


In [62]:
totalfile = sum([len(files) for r, d, files in os.walk("/content/dataset/test/")])
#totalfile = sum([len(files) for r, d, files in os.walk("/content/dataset/train/")])
#totalfile = sum([len(files) for r, d, files in os.walk("/content/dataset/dev/")])


print(totalfile)
count = 0
setData = []
for i in range(1,totalfile+1):
  path = "/content/dataset/" + "test/" + "testdata" +str(i)+".json"
  #path = "/content/dataset/" + "train/" + "traindata" +str(i)+".json"
  #path = "/content/dataset/" + "dev/" + "validdata" +str(i)+".json"
  f = open(path,"r")
  testdata = json.loads(f.read())
  f.close()
  
  arti = splitSent(testdata['article'])
  que = splitSent(testdata['questions'][0])
  setData.append(arti)
  setData.append(que)

setDataEmb = sifEmbeddings(setData,model,alpha)
setDataEmb = list(setDataEmb)
print(len(setDataEmb),len(setDataEmb[0]),count)
l_set = len(setDataEmb)
newData = []
for i in range(0,l_set,2):
  newData.append(np.concatenate([setDataEmb[i],setDataEmb[i+1]]))

print(len(newData),len(newData[0]))
np.savetxt(basePath+'testDataEmbs.csv',newData, delimiter=',',fmt='%.15f')
#np.savetxt(basePath+'trainDataEmbs.csv',newData, delimiter=',',fmt='%.15f')
#np.savetxt(basePath+'validDataEmbs.csv',newData, delimiter=',',fmt='%.15f')

884
1768 200 0
884 400
